In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
import plotly.graph_objs as go

In [68]:
# Use plotly in offline mode
cf.go_offline()

In [69]:
import_data = pd.read_csv('../data/import_data_india.csv')
export_data = pd.read_csv('../data/export_data_india.csv')

In [70]:
print(import_data.info())
print(export_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106985 entries, 0 to 106984
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   country        106985 non-null  object
 1   countrycode    106985 non-null  int64 
 2   hscode         106985 non-null  int64 
 3   commodity      106982 non-null  object
 4   importinlakhs  98628 non-null   object
 5   year           106985 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 4.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114282 entries, 0 to 114281
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   country        114282 non-null  object
 1   countrycode    114282 non-null  int64 
 2   hscode         114282 non-null  int64 
 3   commodity      114282 non-null  object
 4   importinlakhs  109069 non-null  object
 5   year           114282 non-null  int64 
dtypes: int64(3), obj

In [71]:
# Drop Nan values in commodity column as the number of NaNs are very low
import_data.dropna(subset=['commodity'], inplace=True)
export_data.dropna(subset=['commodity'], inplace=True)

# Fill NaN values with 0 as it could mean there was no trade done at all
#TODO: Need to verify this.
import_data['commodity'].fillna(value=0, inplace=True)
export_data['commodity'].fillna(value=0, inplace=True)

In [72]:
apparel = ('ARTICLES OF APPAREL AND CLOTHING ACCESSORIES; NOT KNITTED OR CROCHETED.', 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES; KNITTED OR CORCHETED.', 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIESMADE OF WILD ANIMALS COVERED UNDER WILD LIFE PROTECTION ACT')

import_data['commodity'].replace(to_replace=apparel, value='Apparel', inplace=True)

In [73]:
# import_data[import_data['commodity'].isin(apparel)]['commodity'].value_counts()
import_data[import_data['commodity'] == 'Apparel']['commodity'].value_counts()

Apparel    2016
Name: commodity, dtype: int64

In [74]:
# Convert amount to Crores from lakhs.
# importinlakhs has a weird bug from source so removing comma or semicolon from it
import_data['value'] = import_data['importinlakhs'].str.replace(';', '').str.replace(',', '').astype(float)/100
export_data['value'] = export_data['importinlakhs'].str.replace(';', '').str.replace(',', '').astype(float)/100

In [77]:
# Some of the commodities are similar, merge them into one or many

import_mappings = [('MINERAL FUELS; MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES.', 'Minerals'),
       ('NATURAL OR CULTURED PEARLS;PRECIOUS OR SEMIPRECIOUS STONES,PRE.METALS,CLAD WITH PRE.METAL AND ARTCLS THEREOF;IMIT.JEWLRY;COIN.', 'Gold(Metals)'),
       ('ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS,AND PARTS.', 'Electrical'),
       ('NUCLEAR REACTORS; BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF.', 'Nuclear'),
       ('ORGANIC CHEMICALS', 'PLASTIC AND ARTICLES THEREOF.', 'IRON AND STEEL', 'Chemicals'),
       ('ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLEAVAGE PRODUCTS; PRE. EDIBLE FATS; ANIMAL OR VEGETABLE WAXEX.', 'Edible Oil'),
       ('OPTICAL; PHOTOGRAPHIC CINEMATOGRAPHIC MEASURING, CHECKING PRECISION, MEDICAL OR SURGICAL INST. AND APPARATUS PARTS AND ACCESSORIES THEREOF;', 'Medical Instruments'),
       ('AIRCRAFT; SPACECRAFT, AND PARTS THEREOF.', 'Aircraft'), ('FERTILISERS.', 'Fertilizers'),
       ('MISCELLANEOUS CHEMICAL PRODUCTS.', 'Other Chemicals'),
       ('VEHICLES OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK; AND PARTS AND ACCESSORIES THEREOF.', 'Vehicle Related'),
       ('ARTICLES OF IRON OR STEEL', 'Iron/Steel articles'),
       ('INORGANIC CHEMICALS; ORGANIC OR INORGANIC COMPOUNDS OF PRECIOUS METALS; OF RARE-EARTH METALS, OR RADI. ELEM. OR OF ISOTOPES.', 'Rare Earth Metals')]

export_mappings = [('MINERAL FUELS; MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES.', 'Minerals'),
                         ('NATURAL OR CULTURED PEARLS;PRECIOUS OR SEMIPRECIOUS STONES,PRE.METALS,CLAD WITH PRE.METAL AND ARTCLS THEREOF;IMIT.JEWLRY;COIN.', 'Gold(Metals)'),
                         ('NUCLEAR REACTORS; BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF.', 'Nuclear'),
                         ('ORGANIC CHEMICALS', 'CHEMICALS'),
                         ('VEHICLES OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK; AND PARTS AND ACCESSORIES THEREOF.', 'Vehicle Related'),
                         ('ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS,AND PARTS.', 'Electrical'),
                         ('PHARMACEUTICAL PRODUCTS', 'Pharmaceutical'), ('IRON AND STEEL', 'Iron & Steel'),
                         ('ARTICLES OF APPAREL AND CLOTHING ACCESSORIES; KNITTED OR CORCHETED.', 'Apparel'),
                         ('ARTICLES OF APPAREL AND CLOTHING ACCESSORIES; NOT KNITTED OR CROCHETED.', 'Apparel'),
                         ('FISH AND CRUSTACEANS; MOLLUSCS AND OTHER AQUATIC INVERTABRATES.', 'Aquatic'),
                         ('ARTICLES OF IRON OR STEEL', 'PLASTIC AND ARTICLES THEREOF.', 'Iron & Steel Articles'),
                         ('ALUMINIUM AND ARTICLES THEREOF.', 'Aluminium Articles'), ('CEREALS.', 'Cereals')]

for x in import_mappings:
    import_data.replace(x[0], x[1], inplace=True)

for y in export_mappings:
    import_data.replace(y[0], y[1], inplace=True)

In [78]:
# Drop unwanted columns
columns_to_drop = ['countrycode', 'hscode', 'importinlakhs']
import_data.drop(columns_to_drop, axis=1, inplace=True)
export_data.drop(columns_to_drop, axis=1, inplace=True)

In [79]:
# Check the  data head to see if everything is in place
print(import_data.head())
print(export_data.head())

   country                              commodity  year    value
0  ALGERIA                           KABULI CHANA  2019   0.7771
1  ALGERIA         LENTILS (MOSUR);DRIED AND SHLD  2019   0.0724
2  ALGERIA  DRY DATES SOFT (KHAYZUR OR WET DATES)  2019  10.7641
3  ALGERIA                         KERNELS; H.P.S  2019      NaN
4  ALGERIA  OTHERS; DERIVED FROM VEGETBLE PRODCTS  2019   0.0009
   country                                 commodity  year     value
0  ALGERIA  BONELESS MEAT OF BOVINE ANIMALS ; FROZEN  2019  399.4606
1  ALGERIA                    YELLOWFIN TUNAS FROZEN  2019    5.4880
2  ALGERIA  SKIPJACK OR STRIPE-BELLIED BONITO FROZEN  2019    0.6328
3  ALGERIA                        RIBBON FISH FROZEN  2019       NaN
4  ALGERIA   CROAKERS; GROUPERS AND FLOUNDERS FROZEN  2019    0.0221


In [80]:
# Get top countries and commodities for both import export
top_entries = 15
import_top_commodities_by_value = import_data.groupby('commodity')['value'].sum().sort_values(ascending=False).head(top_entries)
import_top_countries_by_value = import_data.groupby('country')['value'].sum().sort_values(ascending=False).head(top_entries)
export_top_commodities_by_value = export_data.groupby('commodity')['value'].sum().sort_values(ascending=False).head(top_entries)
export_top_countries_by_value = export_data.groupby('country')['value'].sum().sort_values(ascending=False).head(top_entries)

In [81]:
# import_top_commodities_by_value.get_index['ARTICLES OF APPAREL AND CLOTHING ACCESSORIES; NOT KNITTED OR CROCHETED.']

In [82]:
print("Import------------\n", import_top_commodities_by_value)
print("Export------------\n", export_top_commodities_by_value)
print("Import------------\n", import_top_countries_by_value)
print("Export------------\n", export_top_countries_by_value)

Import------------
 commodity
Minerals                         1.948712e+07
Gold(Metals)                     7.012213e+06
Electrical                       6.836522e+06
Nuclear                          5.803272e+06
PLASTIC AND ARTICLES THEREOF.    4.624083e+06
Iron & Steel                     1.412759e+06
Edible Oil                       1.334368e+06
Medical Instruments              1.200535e+06
Aircraft                         1.014820e+06
Fertilizers                      8.767557e+05
Other Chemicals                  7.434184e+05
Vehicle Related                  6.832257e+05
Iron/Steel articles              6.299049e+05
Rare Earth Metals                6.275485e+05
COPPER AND ARTICLES THEREOF.     5.895299e+05
Name: value, dtype: float64
Export------------
 commodity
MINERAL FUELS; MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES.                                                     5.426332e+06
NATURAL OR CULTURED PEARLS;PRECIOUS OR SEMIPRECIOUS STO

In [83]:
# Get only records of top countries and commodities
import_top_country_data = import_data[import_data.country.isin(import_top_countries_by_value.index)]
import_top_commodity_data = import_data[import_data.commodity.isin(import_top_commodities_by_value.index)]
export_top_country_data = export_data[export_data.country.isin(export_top_countries_by_value.index)]
export_top_commodity_data = export_data[export_data.commodity.isin(export_top_commodities_by_value.index)]


In [85]:
print(import_top_commodities_by_value.index)
print("=========================================================")
print(export_top_commodities_by_value.index)

Index(['Minerals', 'Gold(Metals)', 'Electrical', 'Nuclear',
       'PLASTIC AND ARTICLES THEREOF.', 'Iron & Steel', 'Edible Oil',
       'Medical Instruments', 'Aircraft', 'Fertilizers', 'Other Chemicals',
       'Vehicle Related', 'Iron/Steel articles', 'Rare Earth Metals',
       'COPPER AND ARTICLES THEREOF.'],
      dtype='object', name='commodity')
Index(['MINERAL FUELS; MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES.',
       'NATURAL OR CULTURED PEARLS;PRECIOUS OR SEMIPRECIOUS STONES,PRE.METALS,CLAD WITH PRE.METAL AND ARTCLS THEREOF;IMIT.JEWLRY;COIN.',
       'NUCLEAR REACTORS; BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF.',
       'ORGANIC CHEMICALS',
       'VEHICLES OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK; AND PARTS AND ACCESSORIES THEREOF.',
       'ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS,AND PARTS.',
       'PHARMACEUT

In [97]:
usa_china_import = import_top_commodity_data[import_top_commodity_data['country'].isin(['U S A', 'CHINA P RP'])][['commodity', 'country', 'value']]

fig = usa_china_import.sort_values(by='value', ascending=False).\
    groupby(['country', 'commodity']).sum().unstack().\
    iplot(kind='bar', barmode='stack', asFigure=True)

# fig.layout = cf.Layout(height=1000, width=1400, legend=dict(font=dict(size=7), x=2, y=3))
fig.layout.legend = dict(font=dict(size=7), x=2, y=0)
fig.layout.height = 800
fig.layout.width = 1000
# fig.layout.size = 10
fig.show()

In [98]:
usa_china_export = export_top_commodity_data[export_top_commodity_data['country'].isin(['U S A', 'CHINA P RP'])][['commodity', 'country', 'value']]

fig = usa_china_export.sort_values(by='value', ascending=False).\
    groupby(['country', 'commodity']).sum().unstack().\
    iplot(kind='bar', barmode='stack', asFigure=True)

# fig.layout = cf.Layout(height=1000, width=1400, legend=dict(font=dict(size=7), x=2, y=3))
fig.layout.legend = dict(font=dict(size=7), x=2, y=0)
fig.layout.height = 800
fig.layout.width = 1200
# fig.layout.size = 10
fig.show()

In [66]:
# import_top_commodity_data[import_top_commodity_data['country'] == 'CHINA P RP'].groupby(['commodity', 'year']).sum().unstack()


value               \
year                                                       1997         1998   
commodity                                                                      
AIRCRAFT; SPACECRAFT, AND PARTS THEREOF.                80.6369      80.6369   
ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLE...     109.3161     109.3161   
ARTICLES OF IRON OR STEEL                             9965.8349    9965.8349   
ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS TH...  120181.9708  120181.9708   
FERTILISERS.                                         12157.9383   12157.9383   
INORGANIC CHEMICALS; ORGANIC OR INORGANIC COMPO...    4676.9453    4676.9453   
IRON AND STEEL                                        7270.5377    7270.5377   
MINERAL FUELS; MINERAL OILS AND PRODUCTS OF THE...    2603.9932    2603.9932   
MISCELLANEOUS CHEMICAL PRODUCTS.                      7560.6767    7560.6767   
NATURAL OR CULTURED PEARLS;PRECIOUS OR SEMIPREC...     918.6618     918.6618   
NUCLEAR REACTORS; BOILERS, MACHINERY AND MECHAN...   83088.3433   83088.3433   
OPTICAL; PHOTOGRAPHIC CINEMATOGRAPHIC MEASURING...    8349.4619    8349.4619   
ORGANIC CHEMICALS                                    49443.0604   49443.0604   
PLASTIC AND ARTICLES THEREOF.                        16864.3907   16864.3907   
VEHICLES OTHER THAN RAILWAY OR TRAMWAY ROLLING ...    7686.3916    7686.3916   

                                                                              \
year                                                       1999         2000   
commodity                                                                      
AIRCRAFT; SPACECRAFT, AND PARTS THEREOF.                80.6369      80.6369   
ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLE...     109.3161     109.3161   
ARTICLES OF IRON OR STEEL                             9965.8349    9965.8349   
ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS TH...  120181.9708  120181.9708   
FERTILISERS.                                         12157.9383   12157.9383   
INORGANIC CHEMICALS; ORGANIC OR INORGANIC COMPO...    4676.9453    4676.9453   
IRON AND STEEL                                        7270.5377    7270.5377   
MINERAL FUELS; MINERAL OILS AND PRODUCTS OF THE...    2603.9932    2603.9932   
MISCELLANEOUS CHEMICAL PRODUCTS.                      7560.6767    7560.6767   
NATURAL OR CULTURED PEARLS;PRECIOUS OR SEMIPREC...     918.6618     918.6618   
NUCLEAR REACTORS; BOILERS, MACHINERY AND MECHAN...   83088.3433   83088.3433   
OPTICAL; PHOTOGRAPHIC CINEMATOGRAPHIC MEASURING...    8349.4619    8349.4619   
ORGANIC CHEMICALS                                    49443.0604   49443.0604   
PLASTIC AND ARTICLES THEREOF.                        16864.3907   16864.3907   
VEHICLES OTHER THAN RAILWAY OR TRAMWAY ROLLING ...    7686.3916    7686.3916   

                                                                              \
year                                                       2001         2002   
commodity                                                                      
AIRCRAFT; SPACECRAFT, AND PARTS THEREOF.                80.6369      80.6369   
ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLE...     109.3161     109.3161   
ARTICLES OF IRON OR STEEL                             9965.8349    9965.8349   
ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS TH...  120181.9708  120181.9708   
FERTILISERS.                                         12157.9383   12157.9383   
INORGANIC CHEMICALS; ORGANIC OR INORGANIC COMPO...    4676.9453    4676.9453   
IRON AND STEEL                                        7270.5377    7270.5377   
MINERAL FUELS; MINERAL OILS AND PRODUCTS OF THE...    2603.9932    2603.9932   
MISCELLANEOUS CHEMICAL PRODUCTS.                      7560.6767    7560.6767   
NATURAL OR CULTURED PEARLS;PRECIOUS OR SEMIPREC...     918.6618     918.6618   
NUCLEAR REACTORS; BOILERS, MACHINERY AND MECHAN...   83088.3433   83088.3433   
OPTICAL; PHOTOGRAPHIC CINEMATOGRAPHIC MEASURING...    83